In [1]:
from myplt import *
import h5py
import scipy
import scipy.stats as st


# Point Dataset 

In [9]:
class Gaussians:
    def __init__(self, mus, sigmas, wgts):
        self.f = [ 
            st.multivariate_normal(mean=mu, cov=sigma) for mu, sigma in zip(mus, sigmas) ]
        self.wgts = np.array( wgts, dtype=float )
    def __call__(self, x):
        return np.sum( [ w*f.pdf(x) for f, w in zip(self.f, self.wgts) ] )
    
mu1 = [ 4, 4, 4 ]
mu2 = [ -4, -4, -4 ]
mu3 = [ 2, 1, -1 ]
sigma1 = np.diag([1,1,1])
sigma2 = np.diag([1,4,9])
sigma3 = np.array( [[3,1,2],[1,6,1.5],[2,1.5,9]] )
gs = Gaussians( [mu1, mu2, mu3], [sigma1, sigma2, sigma3], [.2, .3, .5] )

f = open('3gaussians.csv', 'w')
f.write('x, y, z, logv, v\n')
dim = 20
for i in range(dim+1):
    for j in range(dim+1):
        for k in range(dim+1):
            x = np.array([i,j,k], dtype=float)
            x = x * 12.0 / dim - 6.0
            v = np.log(gs(x))
            f.write('%.6f, %.6f, %.6f, %.6f, %.6f\n'%(*x, v, np.e**v))
f.close()   

In [8]:
with h5py.File( './elucid_mesh_N100_L39.062500.h5' ) as file:
    dct = { k: np.array(file[k]) for k in file.keys()}

In [16]:
d = np.array([ dct['dens'], dct['x'], dct['y'], dct['z'] ], dtype=float).T
d[:, 0][ d[:, 0] < 0.5 ] = 0.5
d[:, 0] = np.log10( d[:, 0] )
np.savetxt('./elucid_mesh_N100_L39.062500.csv', d, header='dens,x,y,z', comments='', delimiter=',')

# 3-D Game of Life

In [ ]:
np.random.choice()

In [102]:
class GameOfLife:
    def __init__( self, rule=[5,7,6,6], init_live_frac = 0.5, gridsize=(20,20,20) ):
        self.gridsize = np.array(gridsize, dtype=int)
        self.ngrid = self.gridsize.prod()
        self.init_live_frac = init_live_frac
        self.rule = np.array(rule, dtype=int)
        
        self._setup()
        self._get_near()
    def _setup(self):
        d = np.random.choice([1, 0], size=self.ngrid, p=(self.init_live_frac, 1-self.init_live_frac)).reshape(*self.gridsize)
        self.data = np.array(d, dtype=int)
        self._flush_boundary()
    def _flush_boundary(self):
        self.data[:, :, 0] = self.data[:, :, -1] = self.data[:, 0, :] = self.data[:, -1, :] = self.data[0,:,:] = self.data[-1,:,:] = 0
    def _get_near(self):
        near = np.zeros( self.gridsize-2 )
        ni,nj,nk = self.gridsize - 2
        for i in 0,1,2:
            for j in 0,1,2:
                for k in 0,1,2:
                    if i != 0 or j != 0 or k != 0:
                        near += self.data[ i:i+ni, j:j+nj, k:k+nk ]
        self.near = np.zeros( self.gridsize )
        self.near[1:-1, 1:-1, 1:-1] = near
    def step(self):
        me = self.data
        near = self.near
        ei, ej, fi, fj = self.rule
        print('rule ', self.rule, end='\t')
        live = ((me == 1) & (near >= ei) & (near<=ej)) | ((me == 0) & (near >= fi) & (near<=fj))
        print(live.sum(), ' lives')
        self.data[ live ] = 1
        self.data[ live == False ] = 0
        self._get_near()
    def dump(self, fname):
        f = open(fname, 'w')
        f.write('x,y,z,near\n')
        ni,nj,nk=self.gridsize
        for k in range(nk):
            for j in range(nj):
                for i in range(ni):
                    if self.data[i,j,k] == 1:
                        f.write('%d,%d,%d,%d\n'%(i,j,k,self.near[i,j,k]))
        f.close()

In [136]:
gol = GameOfLife([5,8,6,6], .3, (40, 40, 40))
gol.data[11:14, 11:14, 11:14] = 1
gol._get_near()
for i in range(200):
    gol.dump( 'game_of_life/n40.csv.%0.4d'%i )
    gol.step()

rule  [5 8 6 6]	13970  lives
rule  [5 8 6 6]	13221  lives
rule  [5 8 6 6]	13097  lives
rule  [5 8 6 6]	12924  lives
rule  [5 8 6 6]	12697  lives
rule  [5 8 6 6]	12669  lives
rule  [5 8 6 6]	12629  lives
rule  [5 8 6 6]	12538  lives
rule  [5 8 6 6]	12570  lives
rule  [5 8 6 6]	12870  lives
rule  [5 8 6 6]	12699  lives
rule  [5 8 6 6]	12717  lives
rule  [5 8 6 6]	12239  lives
rule  [5 8 6 6]	12731  lives
rule  [5 8 6 6]	12234  lives
rule  [5 8 6 6]	12432  lives
rule  [5 8 6 6]	12181  lives
rule  [5 8 6 6]	12274  lives
rule  [5 8 6 6]	12207  lives
rule  [5 8 6 6]	12685  lives
rule  [5 8 6 6]	12533  lives
rule  [5 8 6 6]	12648  lives
rule  [5 8 6 6]	12567  lives
rule  [5 8 6 6]	12562  lives
rule  [5 8 6 6]	12455  lives
rule  [5 8 6 6]	12581  lives
rule  [5 8 6 6]	12457  lives
rule  [5 8 6 6]	12427  lives
rule  [5 8 6 6]	12100  lives
rule  [5 8 6 6]	12420  lives
rule  [5 8 6 6]	12358  lives
rule  [5 8 6 6]	12354  lives
rule  [5 8 6 6]	12088  lives
rule  [5 8 6 6]	12356  lives
rule  [5 8 6 6